In [1]:
import pandas as pd
from sklearn.datasets import load_iris
from factor_analyzer import FactorAnalyzer
from factor_analyzer.factor_analyzer import calculate_bartlett_sphericity
from factor_analyzer.factor_analyzer import calculate_kmo
import datetime

In [2]:
excel_file="/home/jian/Projects/Saatva/MMM/URL_models/input/URL_SM_input_tabs_data_JL_2019-05-06.xlsx"
excel_file=pd.ExcelFile(excel_file)
excel_file.sheet_names

tab_list=[x for x in excel_file.sheet_names if "Stat" not in x]

In [3]:
df_data=excel_file.parse('Home Page')
df_ivs=df_data.copy()
df_ivs.drop(['date','trans_net','sales_net', 'Users', 'Sessions','ga:goal11Completions','Time on Page','Transactions'],axis=1,inplace=True)



In [4]:
rotation_methods_list=['varimax','promax','oblimin','oblimax','quartimin','quartimax','equamax']


In [5]:
chi_square_value,p_value=calculate_bartlett_sphericity(df_ivs)
chi_square_value, p_value

(10386.769576877074, 0.0)

In [6]:
kmo_by_variable,kmo_all_ivs=calculate_kmo(df_ivs)

print("kmo_all_ivs:",kmo_all_ivs)
print("kmo_by_variable:",kmo_by_variable)


kmo_all_ivs: 0.846048719485
kmo_by_variable: [ 0.87213852  0.87526528  0.89824781  0.81614876  0.74854401  0.80875286
  0.5720416   0.90397856  0.89417893  0.8610752   0.69680254  0.9268563
  0.74850947  0.77042943  0.92060286  0.83679093  0.89013907  0.8121889
  0.83850682  0.84143357  0.71010519  0.89058918  0.81752437  0.91968752
  0.77529737  0.85952808  0.88547238  0.84796858  0.85545715  0.84121155
  0.72158802  0.87468988  0.56013039  0.81743091  0.91050535  0.6943214
  0.75851238  0.9001076   0.90349207  0.86015329  0.87106576  0.79220214
  0.79302715  0.82963755  0.6764021   0.90119048  0.85104514  0.87406245
  0.85242156  0.86931145  0.84435511]


In [7]:
factors_num=30
i_counter=0
columns_list=["method","factors_set",'original_eigen_values','common_factor_eigen_values','nth_original_factor_at_least_1','nth_common_factor_at_least_1']
df_method_eigen_values=pd.DataFrame(columns=columns_list)

dict_method_factor_num={}
for method in rotation_methods_list:

    fa = FactorAnalyzer(rotation=method,n_factors=factors_num)
    fa.fit(df_ivs)
    original_eigen_values,common_factor_eigen_values=fa.get_eigenvalues()

    df=pd.DataFrame({"method":method,"factors_set":factors_num,"original_eigen_values":[original_eigen_values],"common_factor_eigen_values":[common_factor_eigen_values],'nth_original_factor_at_least_1':len([x for x in original_eigen_values if x >1]),'nth_common_factor_at_least_1':len([x for x in common_factor_eigen_values if x >1])},index=[i_counter])
    i_counter+=1
    df_method_eigen_values=df_method_eigen_values.append(df)
    print(method,datetime.datetime.now())
    dict_method_factor_num.update({method:len([x for x in common_factor_eigen_values if x >1])})
df_method_eigen_values=df_method_eigen_values[columns_list]

varimax 2019-05-07 18:09:14.133319
promax 2019-05-07 18:09:16.483023
oblimin 2019-05-07 18:09:19.157491
oblimax 2019-05-07 18:09:25.101820
quartimin 2019-05-07 18:09:27.750970
quartimax 2019-05-07 18:09:30.730852
equamax 2019-05-07 18:09:33.700306


In [8]:
dict_method_factor_num

{'equamax': 8,
 'oblimax': 8,
 'oblimin': 7,
 'promax': 9,
 'quartimax': 8,
 'quartimin': 7,
 'varimax': 8}

In [9]:
'''
# decrypt the algebra
# varima
method="varimax"
fa = FactorAnalyzer(rotation=method,n_factors=dict_method_factor_num[method])
fa.fit(df_ivs)
df_commnu_uniqueness=pd.DataFrame({"iv":df_ivs.columns.tolist(),"communalities":fa.get_communalities(),"uniquenesses":fa.get_uniquenesses()})
df_factor_variance=pd.DataFrame({"factor":[x for x in range(dict_method_factor_num[method])],
                                 "variance":fa.get_factor_variance()[0],
                                 "proportional_variance":fa.get_factor_variance()[1],
                                 "cumulative_variances":fa.get_factor_variance()[2]})
df_commnu_uniqueness=df_commnu_uniqueness[['iv','communalities','uniquenesses']]
df_factor_variance=df_factor_variance[['factor','variance','proportional_variance','cumulative_variances']]
df_loadings=pd.DataFrame(fa.loadings_,index=[df_ivs.columns.tolist()])
df_loadings=df_loadings.reset_index()
df_transform_data=pd.DataFrame(fa.transform(df_ivs))
df_transform_data.index=df_data['date'].tolist()
df_transform_data=df_transform_data.reset_index()

if fa.structure_ is not None:
    structure = fa.structure_
else:
    structure = fa.loadings_
    
weights = np.linalg.solve(fa.corr_, structure)

if isinstance(df_ivs, pd.DataFrame):
    X = df_ivs.copy().values
else:
    X = df_ivs.copy()



# see if we saved the original mean and std
if fa.mean_ is None or fa.std_ is None:
    warnings.warn('Could not find original mean and standard deviation; using'
                  'the mean and standard deviation from the current data set.')
    mean = np.mean(X, axis=0)
    std = np.std(X, axis=0)
else:
    mean = fa.mean_
    std = fa.std_

# get the scaled data
X_scale = (X - mean) / std
np.dot(X_scale, weights)
'''

'\n# decrypt the algebra\n# varima\nmethod="varimax"\nfa = FactorAnalyzer(rotation=method,n_factors=dict_method_factor_num[method])\nfa.fit(df_ivs)\ndf_commnu_uniqueness=pd.DataFrame({"iv":df_ivs.columns.tolist(),"communalities":fa.get_communalities(),"uniquenesses":fa.get_uniquenesses()})\ndf_factor_variance=pd.DataFrame({"factor":[x for x in range(dict_method_factor_num[method])],\n                                 "variance":fa.get_factor_variance()[0],\n                                 "proportional_variance":fa.get_factor_variance()[1],\n                                 "cumulative_variances":fa.get_factor_variance()[2]})\ndf_commnu_uniqueness=df_commnu_uniqueness[[\'iv\',\'communalities\',\'uniquenesses\']]\ndf_factor_variance=df_factor_variance[[\'factor\',\'variance\',\'proportional_variance\',\'cumulative_variances\']]\ndf_loadings=pd.DataFrame(fa.loadings_,index=[df_ivs.columns.tolist()])\ndf_loadings=df_loadings.reset_index()\ndf_transform_data=pd.DataFrame(fa.transform(df_iv

In [10]:
fa.is_corr_matrix

False

In [11]:

'''
# factors_num=12
i_counter=0

dic_df_evaluation={}
for method in rotation_methods_list:
    fa = FactorAnalyzer(rotation=method,n_factors=dict_method_factor_num[method])
    fa.fit(df_ivs)
    df_commnu_uniqueness=pd.DataFrame({"iv":df_ivs.columns.tolist(),"communalities":fa.get_communalities(),"uniquenesses":fa.get_uniquenesses()})
    df_factor_variance=pd.DataFrame({"factor":[x for x in range(dict_method_factor_num[method])],
                                     "variance":fa.get_factor_variance()[0],
                                     "proportional_variance":fa.get_factor_variance()[1],
                                     "cumulative_variances":fa.get_factor_variance()[2]})
    df_commnu_uniqueness=df_commnu_uniqueness[['iv','communalities','uniquenesses']]
    df_factor_variance=df_factor_variance[['factor','variance','proportional_variance','cumulative_variances']]
    df_loadings=pd.DataFrame(fa.loadings_,index=[df_ivs.columns.tolist()])
    df_loadings=df_loadings.reset_index()
    df_transform_data=pd.DataFrame(fa.transform(df_ivs))
    df_transform_data.index=df_data['date'].tolist()
    df_transform_data=df_transform_data.reset_index()
    dic_df_evaluation.update({method:{"df_commnu_uniqueness":df_commnu_uniqueness,"df_factor_variance":df_factor_variance,"df_loadings":df_loadings,"transform_data":df_transform_data}})

##########
writer=pd.ExcelWriter("/home/jian/Projects/Saatva/MMM/URL_models/input/EFA_HomePage_evaluation_JL_"+str(datetime.datetime.now().date())+".xlsx",engine="xlsxwriter")
pd.DataFrame({"kmo_by_variable":kmo_by_variable}).to_excel(writer,"kmo_by_variable",index=False)
df_method_eigen_values.to_excel(writer,"eigen_values_by_rotation",index=False)
for method in dic_df_evaluation.keys():
    for key_df in dic_df_evaluation[method].keys():
        dic_df_evaluation[method][key_df].to_excel(writer,method+"|"+key_df.replace("df_",""),index=False)

writer.save()


'''

'\n# factors_num=12\ni_counter=0\n\ndic_df_evaluation={}\nfor method in rotation_methods_list:\n    fa = FactorAnalyzer(rotation=method,n_factors=dict_method_factor_num[method])\n    fa.fit(df_ivs)\n    df_commnu_uniqueness=pd.DataFrame({"iv":df_ivs.columns.tolist(),"communalities":fa.get_communalities(),"uniquenesses":fa.get_uniquenesses()})\n    df_factor_variance=pd.DataFrame({"factor":[x for x in range(dict_method_factor_num[method])],\n                                     "variance":fa.get_factor_variance()[0],\n                                     "proportional_variance":fa.get_factor_variance()[1],\n                                     "cumulative_variances":fa.get_factor_variance()[2]})\n    df_commnu_uniqueness=df_commnu_uniqueness[[\'iv\',\'communalities\',\'uniquenesses\']]\n    df_factor_variance=df_factor_variance[[\'factor\',\'variance\',\'proportional_variance\',\'cumulative_variances\']]\n    df_loadings=pd.DataFrame(fa.loadings_,index=[df_ivs.columns.tolist()])\n    df

In [12]:
method="varimax"

fa = FactorAnalyzer(rotation=method,n_factors=dict_method_factor_num[method])
fa.fit(df_ivs)
df_commnu_uniqueness=pd.DataFrame({"iv":df_ivs.columns.tolist(),"communalities":fa.get_communalities(),"uniquenesses":fa.get_uniquenesses()})
df_factor_variance=pd.DataFrame({"factor":[x for x in range(dict_method_factor_num[method])],
                                 "variance":fa.get_factor_variance()[0],
                                 "proportional_variance":fa.get_factor_variance()[1],
                                 "cumulative_variances":fa.get_factor_variance()[2]})
df_commnu_uniqueness=df_commnu_uniqueness[['iv','communalities','uniquenesses']]
df_factor_variance=df_factor_variance[['factor','variance','proportional_variance','cumulative_variances']]
df_loadings=pd.DataFrame(fa.loadings_,index=[df_ivs.columns.tolist()])
df_loadings=df_loadings.reset_index()
df_transform_data=pd.DataFrame(fa.transform(df_ivs))
df_transform_data.index=df_data['date'].tolist()
df_transform_data=df_transform_data.reset_index()

In [13]:
df_data.head(2)

,date,sales_net,trans_net,Users,Sessions,ga:goal11Completions,Time on Page,Transactions,www.saatvamattress.com/made-in-america.html,www.saatvamattress.com/foundations.html,...,www.saatvamattress.com/saatva-reviews,www.saatvamattress.com/mattress-for-back-pain-relief,www.saatvamattress.com/saatva-mattress.html,www.saatvamattress.com/sitemap,www.saatvamattress.com/mattress-comparison-chart.html,www.saatvamattress.com/saatva-mattress-reviews,www.saatvamattress.com/our-comfort-zone.html,www.saatvamattress.com/the-mattresses.html,www.saatvamattress.com,www.saatvamattress.com/shop-pla
0,2018-01-01,330007.80,232,27488,31733,727,3859793,185,5,0,...,0,294,1228,2,50,153,232,442,22331,429
1,2018-01-02,316132.57,227,28227,32226,677,3670182,156,11,0,...,0,309,1204,0,44,132,181,500,21763,393


In [14]:
DVs=df_data[['date','trans_net','Users','Sessions','ga:goal11Completions']]
DVs['Trans_Per_Usr']=DVs['trans_net']/DVs['Users']

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [15]:
varimax_MMM_input=df_transform_data.rename(columns={"index":"date"})

for col in range(8):
    varimax_MMM_input=varimax_MMM_input.rename(columns={col:"Factor_"+str(col)})
varimax_MMM_input=pd.merge(DVs,varimax_MMM_input,on="date")
varimax_MMM_input.to_csv("/home/jian/Projects/Saatva/MMM/URL_models/input/EFA_Varimax_HomePage_MMM_JL_"+str(datetime.datetime.now().date())+".csv",index=False)
                         
                         
                         